# LFS tool

This tool automates the process of setting up **git-lfs** on a git repository. 

Run the code cell below to select a **git** repository. Press **`select`** to pull up the file chooser; then press **`select`** again. You should see a button that says **`change`** and your repository to the right in green text.

In [ ]:
from ipywidgets import widgets
from ipyfilechooser import FileChooser
import os
#Do this once or it gets confusing
try:
    fc
except:
    done = True
    # Create and display a FileChooser widget
    fc = FileChooser(os.path.expanduser("~/"))
    fc.show_only_dirs = True
    fc.title = '<b>Choose a repository:</b>'
    display(fc)

### Specify file patterns

Run the next cell to see a list of file patterns considered "big"; remove or add patterns to the list:

In [ ]:
#Generate the widget only once
try:
    w
except:
    w = widgets.Textarea(
        value='data/**\n*.wav\n*.png\n*.jpg\n*.mp3\n*.mp4\n*.webm\n*.ogg\n*.pt\n*.h5',
        placeholder='Type something',
        description='LFS patterns:',
        disabled=False,
        layout=widgets.Layout(height="172px")
    )
    display(w)

### Save the lfs patterns
The file patterns are saved in a hidden file called 
`.gitattributes`

In [ ]:
path = fc.selected + ".gitattributes"
print ("Creating file", path)
f = open(path, 'w')
for row in w.value.split("\n"):
    f.write(row + " filter=lfs diff=lfs merge=lfs -text\n")
f.close()
cmd = f"cd {fc.selected}; git add .gitattributes"
print (f"Adding to project:\n{cmd}")
if os.system(cmd):
    raise Exception("error")

### Point to an **lfs** server

Cloudburst provides a free, public LFS server for specific uses (see the TOS).
Anyone with the LFS pointer can read your data, but there is no way to overwrite files.

Note: we are using your local folder name as the remote lfs repository name. If your repository has a different name, edit the url accordingly.

Edit to specify another server, or delete to use default provider's server:


In [ ]:
try:
    t
except:
    url = fc.selected.strip("/")
    parts = url.split("/")
    user = parts[-2]
    if user.find("jupyter-") == 0:
        user = user[8:]
    repo = parts[-1]
    url = f"https://lfs.datahub.host/{user}/{repo}"
    t = widgets.Text(
        value=url,
        placeholder='Type something',
        description='LFS server:',
        layout=widgets.Layout(width="420px"),
        disabled=False
    )
    display(t)

Contact us at lfsinfo@cloudburst.host to discuss private lfs storage solutions.

### Save the server location

Linus really digs hidden files! This one is called `.lfsconfig`:

In [ ]:
if t.value:
    path2 = fc.selected + ".lfsconfig"
    print ("Setting LFS server to", t.value)
    f = open(path2, 'w')
    f.write("[lfs]\n  url = " + t.value)
    f.close()
    cmd = f"cd {fc.selected}; git add .lfsconfig"
    print (f"Adding to project:\n{cmd}")
    if os.system(cmd):
        raise Exception("error")
else:
    print ("Leaving lfs url blank; git-lfs will use your git server's associated lfs server")

### Update big files to the lfs lifestyle

In [ ]:
#So intuitive! And that documentation...
cmd = f"cd {fc.selected}; git add --renormalize ."
print (f"Adding to project:\n{cmd}")
if os.system(cmd):
    raise Exception("error")

### Commit the changes
We've modified your big files to be lfs-compliant, and created two new system files that need to be saved to the repository. To keep these changes, run a command like:

    git commit -m "Set up LFS"
    
Good luck!